In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm

from waymo_open_dataset import dataset_pb2 as open_dataset


2026-01-16 19:38:24.499690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-16 19:38:23.636783: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-16 19:38:23.642586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-16 19:38:23.642611: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
TRAIN_TFRECORD = "/home/kakani/waymo_project/1_data/tfrecords/training"
VAL_TFRECORD   = "/home/kakani/waymo_project/1_data/tfrecords/validation"

RAW_IMG_OUT = "/home/kakani/waymo_project/1_data/extracted/images_raw"
RAW_LBL_OUT = "/home/kakani/waymo_project/1_data/extracted/labels_raw"

os.makedirs(RAW_IMG_OUT, exist_ok=True)
os.makedirs(RAW_LBL_OUT, exist_ok=True)

print("✅ Paths Ready")


✅ Paths Ready


In [3]:
train_files = sorted([os.path.join(TRAIN_TFRECORD, f) for f in os.listdir(TRAIN_TFRECORD) if f.endswith(".tfrecord")])
val_files   = sorted([os.path.join(VAL_TFRECORD, f) for f in os.listdir(VAL_TFRECORD) if f.endswith(".tfrecord")])

TRAIN_FILE = train_files[0]
VAL_FILE   = val_files[0]

print("✅ TRAIN_FILE:", TRAIN_FILE)
print("✅ VAL_FILE:", VAL_FILE)


✅ TRAIN_FILE: /home/kakani/waymo_project/1_data/tfrecords/training/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord
✅ VAL_FILE: /home/kakani/waymo_project/1_data/tfrecords/validation/segment-10203656353524179475_7625_000_7645_000_with_camera_labels.tfrecord


In [4]:
WAYMO_TO_YOLO = {
    1: 0,   # Vehicle
    2: 1,   # Pedestrian
    3: 2,   # Sign
    4: 3    # Cyclist
}

YOLO_CLASS_NAMES = {
    0: "VEHICLE",
    1: "PEDESTRIAN",
    2: "SIGN",
    3: "CYCLIST"
}
print("✅ Label map ready")


✅ Label map ready


In [5]:
def save_yolo_label(txt_path, labels):
    with open(txt_path, "w") as f:
        for row in labels:
            f.write(" ".join(map(str, row)) + "\n")


In [6]:
def extract_from_tfrecord(tfrecord_path, prefix, max_frames=20):
    dataset = tf.data.TFRecordDataset(tfrecord_path, compression_type='')

    records = []
    frame_id = 0

    for data in tqdm(dataset.take(max_frames), desc=f"Extracting {prefix}"):
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))

        # only FRONT camera
        target_camera = open_dataset.CameraName.FRONT

        front_img = None
        for img in frame.images:
            if img.name == target_camera:
                front_img = img
                break

        if front_img is None:
            continue

        # decode image using OpenCV
        img_np = np.frombuffer(front_img.image, np.uint8)
        img_cv = cv2.imdecode(img_np, cv2.IMREAD_COLOR)

        if img_cv is None:
            continue

        h, w, _ = img_cv.shape

        # output filenames
        img_name = f"{prefix}_frame_{frame_id:04d}.jpg"
        lbl_name = f"{prefix}_frame_{frame_id:04d}.txt"

        img_path = os.path.join(RAW_IMG_OUT, img_name)
        lbl_path = os.path.join(RAW_LBL_OUT, lbl_name)

        cv2.imwrite(img_path, img_cv)

        # Extract labels
        yolo_labels = []
        for cam_lbl in frame.camera_labels:
            if cam_lbl.name == target_camera:
                for label in cam_lbl.labels:
                    if label.type not in WAYMO_TO_YOLO:
                        continue

                    cls = WAYMO_TO_YOLO[label.type]
                    box = label.box

                    # Waymo box coords
                    cx, cy = box.center_x, box.center_y
                    bw, bh = box.length, box.width

                    # YOLO normalize
                    x = cx / w
                    y = cy / h
                    bw_n = bw / w
                    bh_n = bh / h

                    yolo_labels.append([cls, x, y, bw_n, bh_n])

                    # for CSV
                    records.append([img_name, YOLO_CLASS_NAMES[cls], cx-bw/2, cy-bh/2, cx+bw/2, cy+bh/2])

        save_yolo_label(lbl_path, yolo_labels)

        print(f"✅ Saved {img_name} + {lbl_name} | boxes={len(yolo_labels)}")
        frame_id += 1

    return records


In [7]:
train_records = extract_from_tfrecord(TRAIN_FILE, "train", max_frames=30)
val_records   = extract_from_tfrecord(VAL_FILE, "val", max_frames=20)

print("✅ Extraction completed!")


2026-01-16 19:39:46.805646: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2026-01-16 19:39:46.805923: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kakani/waymo310/lib/python3.10/site-packages/cv2/../../lib64:
2026-01-16 19:39:46.806024: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/kakani/waymo310/lib/python3.10/site-packages/cv2/../../lib64:
2026-01-16 19:39:46.806104: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Cou

✅ Saved train_frame_0000.jpg + train_frame_0000.txt | boxes=5
✅ Saved train_frame_0001.jpg + train_frame_0001.txt | boxes=5
✅ Saved train_frame_0002.jpg + train_frame_0002.txt | boxes=6


Extracting train: 5it [00:00, 11.51it/s]

✅ Saved train_frame_0003.jpg + train_frame_0003.txt | boxes=7
✅ Saved train_frame_0004.jpg + train_frame_0004.txt | boxes=7
✅ Saved train_frame_0005.jpg + train_frame_0005.txt | boxes=7


Extracting train: 9it [00:00, 12.66it/s]

...

✅ Saved val_frame_0010.jpg + val_frame_0010.txt | boxes=25
✅ Saved val_frame_0011.jpg + val_frame_0011.txt | boxes=24
✅ Saved val_frame_0012.jpg + val_frame_0012.txt | boxes=24


Extracting val: 17it [00:01, 14.59it/s]

✅ Saved val_frame_0013.jpg + val_frame_0013.txt | boxes=25
✅ Saved val_frame_0014.jpg + val_frame_0014.txt | boxes=26
✅ Saved val_frame_0015.jpg + val_frame_0015.txt | boxes=26
✅ Saved val_frame_0016.jpg + val_frame_0016.txt | boxes=26


Extracting val: 20it [00:01, 14.18it/s]

✅ Saved val_frame_0017.jpg + val_frame_0017.txt | boxes=25
✅ Saved val_frame_0018.jpg + val_frame_0018.txt | boxes=24
✅ Saved val_frame_0019.jpg + val_frame_0019.txt | boxes=24
✅ Extraction completed!


In [8]:
df = pd.DataFrame(train_records + val_records,
                  columns=["image","class","xmin","ymin","xmax","ymax"])

csv_path = "/home/kakani/waymo_project/1_data/extracted/annotations.csv"
df.to_csv(csv_path, index=False)

print("✅ CSV saved:", csv_path)
df.head()


✅ CSV saved: /home/kakani/waymo_project/1_data/extracted/annotations.csv


,image,class,xmin,ymin,xmax,ymax
0,train_frame_0000.jpg,VEHICLE,1346.234085,437.691870,1586.869875,547.588530
1,train_frame_0000.jpg,VEHICLE,1019.702055,458.850135,1198.442025,614.852865
2,train_frame_0000.jpg,VEHICLE,1150.756980,471.481935,1212.652800,566.852025
3,train_frame_0000.jpg,VEHICLE,991.912095,467.376600,1020.333645,506.535180
4,train_frame_0000.jpg,VEHICLE,1199.389410,540.009450,1560.658890,667.590630


In [9]:
print(len(os.listdir(RAW_IMG_OUT)), "images")
print(len(os.listdir(RAW_LBL_OUT)), "labels")


50 images
50 labels
